In [1]:
import numpy as np
import torch
import matplotlib.pyplot as plt
import pyvista as pv
from pol.utils.validation.scene_saver import load_scenes, count_h5_keys, find_max_h5_key
from pol.utils.path import PathHelper
from pol.utils.validation.scene_renderer import render_arrows, render_plane, \
    render_tri_mesh

pv.start_xvfb()
pv.set_plot_theme('document')
import matplotlib_inline.backend_inline
matplotlib_inline.backend_inline.set_matplotlib_formats('pdf')
import matplotlib
use_tex = matplotlib.checkdep_usetex(True)
if use_tex:
    plt.rcParams['text.usetex'] = True
plt.rcParams.update({'font.size': 28})

In [2]:
prob = 'mcb_dgcnn' # 'k8_circle'
path_helper = PathHelper('../../tests/symmetry_detection/')

all_methods = ['pd_hlr', 'gol_single_res_hlr',
               'pol_res_dp_mot_hor10', #'pol_res_mot', 'pol_res_lot', 'pol_res_hmot', 'pol_res_mlot', 'pol_res_hot',
               ]
              #]
gt_method = 'pd_mlr'
candidate_methods = all_methods[1:]
#view_box = [[-5, 5], [-5, 5]]
method_itr = {
    'pol_res_mot': 99,
    'gol_res_hlr': 99,
}


cmap = plt.cm.get_cmap('hsv', len(all_methods)+1)
colors = {all_methods[i]: cmap(i) for i in range(len(all_methods))}

def convert_method_to_label(method):
    if method.startswith('gol'):
        return 'GOL'
    if method.startswith('pol'):
        return 'POL'
    if method.startswith('pd'):
        return 'PD'
    return 'Unknown'

In [3]:
def vis1(compact=False):
    scenes = load_scenes('../../tests/symmetry_detection/scenes/mcb_dgcnn/pol_res_dp_mot_hor10/step-100000.h5',
                    unroll=False)

    eval_itrs = [0, 1, 2, 5, 10, 15, 20]
    #mesh_ids = [6, 114, 204, 257, 324, 410, 637, 645, 656, 904, 1017, 1366, 1489, 1519, 2763, 3043] # [0, 100, 200, 400, 800]
    mesh_ids = [904, 645, 410, 204, 257, 324, 656, 637, 1489, 1519, 2763, 3043, 1303, 6, 114, 1366]
    fig_size = 600
    camera = pv.Camera()
    camera.position = (3, 3, 3)
    camera.focal_point = (0, 0, 0)
    
    if compact:
        num_row = 2
        num_col = 8
    else:
        num_row = len(mesh_ids)
        num_col = len(eval_itrs)
    plotter = pv.Plotter(shape=(num_row, num_col), 
                     window_size=(fig_size*num_col, fig_size*num_row),
                         multi_samples=8,
                         border=False,
                        off_screen=True)
    plotter.enable_anti_aliasing()
    
    for r in range(num_row):
        for c in range(num_col):
            if compact:
                mesh_id = mesh_ids[r * num_col + c]
                itr = eval_itrs[-1]
            else:
                mesh_id = mesh_ids[r]
                itr = eval_itrs
            
            scene = scenes[mesh_id]
            mesh = scene['mesh']
            plotter.subplot(r, c)
            plotter.camera = camera
            
            render_tri_mesh(plotter, mesh['vertices'][:], mesh['faces'][:], opacity=0.8)
            tmp = scene[f'itr_{itr}']
            R = tmp['reflections'][:]
            N = R[:, :3]
            I = R[:, 3]
            C = N * np.expand_dims(I, -1)
            
            sdf_loss = tmp['losses'][:]
            render_arrows(plotter, C, N+C, weight=sdf_loss, threshold=1e100, weight_clip=0.05, line_width=4.0,
                          show_endpoints=False,
                          show_scalar_bar=False,
                         cmap='cool_r')

    #plotter.show_axes()
    #plotter.show(screenshot='result.png')
    
    import imageio
    image = plotter.screenshot(None, return_img=True)
    if compact:
        imageio.imwrite('result_cpt.png', image)
    else:
        imageio.imwrite('result.png', image)

#vis1(compact=True)

In [4]:
def vis2():
    from pol.utils.plotting import WitnessMetricsPlotter
    plotter = WitnessMetricsPlotter(path_helper, problem=prob,
                                    gt_method=gt_method, 
                                    candidate_methods=candidate_methods)
    fig = plotter.plot(shortname_fn=convert_method_to_label,
                      colors=colors)
    
vis2()

<Figure size 2880x360 with 4 Axes>